# Object Detection Model Benchmarking

## In this notebook, we will use the modularized project to benchmark object detection models and visualize the results.

In [ ]:
# Clone the repository

!git clone https://github.com/M-S-KHAN/VITs-Benchmarking.git
%cd VITs-Benchmarking

In [1]:
# Then, let's install the necessary libraries. You can skip this step if you already have them installed.

%pip install torch transformers torchvision pillow matplotlib seaborn tqdm

You should consider upgrading via the '/Users/sajawal/developer/msc_dissertation/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from data.data_loader import DataLoader
from utils.utils import load_benchmark_data
from benchmarking.benchmark import run_benchmark
from benchmarking.visualization import VisualizationUtils
from config import MODELS_TO_BENCHMARK
import logging
import json

/Users/sajawal/developer/msc_dissertation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sajawal/developer/msc_dissertation/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Configure logging to display informative messages during the benchmarking process.

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [5]:
# Load the benchmark data. If previous benchmark results exist, load them; otherwise, load the data using the `DataLoader` class.

existing_data = load_benchmark_data()
if existing_data:
    benchmark_data = existing_data
else:
    data_loader = DataLoader()
    benchmark_data = data_loader.load_benchmark_data()

In [6]:
# Run the benchmarks for each model specified in the `MODELS_TO_BENCHMARK` list.

all_results = []
for model_type, model_name in MODELS_TO_BENCHMARK:
    print(f"Starting benchmark for {model_name}")
    try:
        print(f'{model_type} {model_name}')
        result = run_benchmark(model_type, model_name, benchmark_data)
        all_results.append(result)
    except Exception as e:
        logger.error(f"Error benchmarking {model_name}: {e}")

2024-09-26 21:50:36,740 - INFO - Starting benchmark for google/owlvit-base-patch32
2024-09-26 21:50:36,746 - ERROR - Error benchmarking google/owlvit-base-patch32: Unsupported model type: OWL-ViT
2024-09-26 21:50:36,749 - INFO - Starting benchmark for facebook/detr-resnet-50
2024-09-26 21:50:36,751 - INFO - Initializing DETR model: facebook/detr-resnet-50 on device: cpu


Starting benchmark for google/owlvit-base-patch32
OWL-ViT google/owlvit-base-patch32
Starting benchmark for google/owlvit-base-patch32
Starting benchmark for facebook/detr-resnet-50
DETR facebook/detr-resnet-50
Starting benchmark for facebook/detr-resnet-50


/Users/sajawal/developer/msc_dissertation/.venv/lib/python3.9/site-packages/transformers/models/detr/feature_extraction_detr.py:38: FutureWarning: The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.
  warnings.warn(
2024-09-26 21:50:37,516 - INFO - Loading pretrained weights from Hugging Face hub (timm/resnet50.a1_in1k)
2024-09-26 21:50:37,640 - INFO - [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2024-09-26 21:50:37,870 - INFO - Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tr

KeyboardInterrupt: 

In [ ]:
# Display a comparison table of the average metrics for each benchmarked model.

print("\nComparison of Average Metrics:")
metrics = ["mean_iou", "ap_50", "ap_75", "avg_inference_time"]
print(f"{'Model':<30} " + " ".join(f"{m:>12}" for m in metrics))
for result in all_results:
    model_name = result["model_name"].split("/")[-1]
    metrics_values = [f"{result['avg_metrics'][m]:.4f}" for m in metrics]
    print(f"{model_name:<30} " + " ".join(f"{v:>12}" for v in metrics_values))

In [ ]:
# Save the detailed benchmark results to a JSON file.

with open('vision_transformer_benchmark_results.json', 'w') as f:
    json.dump(all_results, f, indent=2)
logger.info("Detailed results saved to vision_transformer_benchmark_results.json")

In [ ]:
# Create various comparison graphs to visualize the benchmark results.

VisualizationUtils.create_comparison_graphs(all_results)